## Train the dataset


Lets train the dataset. We will split the dataset into 80% train data and the rest 20% test data

In [ ]:
import pandas as pd
import time
import sagemaker

In [ ]:
# read the prepared data from S3. Enter any of the Results processed file S3 location
bucket = "bucket_name"
source = 's3://sagemaker-us-east-1-367858208265/Results/DataWrangler/output_1631272206/part-00000-d3369d58-6799-4d9f-91bd-0f0159be50b4-c000.csv'

In [ ]:
train = dataset.sample(frac=0.80, random_state=0)
test = dataset.drop(train.index)

In [ ]:
train.to_csv("train.csv", index=False)
test.to_csv("test.csv", index=False)
dataset.to_csv("dataset.csv", index=True)

Write train, test data to S3


In [ ]:
s3_client.upload_file(
    Filename="train.csv", Bucket=bucket, Key=f"Results/DataSet/train/train.csv"
)
s3_client.upload_file(Filename="test.csv", Bucket=bucket, Key=f"Results/DataSet/test/test.csv")

In [ ]:
train.head(5)

In [ ]:
test.head(5)

## Train a model using XGBoost

Once the training and test datasets have been persisted in S3, you can start training a model by defining which SageMaker Estimator you’d like to use. For this guide, you will use the XGBoost Open Source Framework to train your model. This estimator is accessed via the SageMaker SDK, but mirrors the open source version of the XGBoost Python package. Any functioanlity provided by the XGBoost Python package can be implemented in your training script.

In [ ]:
from sagemaker.debugger import Rule, rule_configs
from sagemaker.session import TrainingInput

sess = sagemaker.Session()
bucket = sess.default_bucket()  
s3_output_location='s3://{}/{}/{}'.format(bucket, "Results/", 'xgboost_model')

container=sagemaker.image_uris.retrieve("xgboost", region, "1.2-1")
print(container)

xgb_model=sagemaker.estimator.Estimator(
    image_uri=container,
    role=role,
    instance_count=1,
    instance_type='ml.m4.xlarge',
    train_volume_size=5,
    output_path=s3_output_location,
    sagemaker_session=sagemaker.Session(),
    rules=[Rule.sagemaker(rule_configs.create_xgboost_report())]
)

### Set the hyperparameters
These are the parameters which will be sent to our training script in order to train the model. Although they are all defined as “hyperparameters” here, they can encompass XGBoost’s Learning Task Parameters, Tree Booster Parameters, or any other parameters you’d like to configure for XGBoost.

In [ ]:
xgb_model.set_hyperparameters(
    max_depth = 5,
    eta = 0.2,
    gamma = 4,
    min_child_weight = 6,
    subsample = 0.7,
    objective = "binary:logistic",
    num_round = 1000
)

### Create and fit the estimator
Use the TrainingInput class to configure a data input flow for training. The following example code shows how to configure TrainingInput objects to use the training and validation datasets you uploaded to Amazon S3

In [ ]:
from sagemaker.session import TrainingInput

train_input = TrainingInput(
    "s3://{}/{}/{}".format(bucket, 'Results', "DataSet/train/train.csv"), content_type="csv"
)
validation_input = TrainingInput(
    "s3://{}/{}/{}".format(bucket, 'Results', "DataSet/test/test.csv"), content_type="csv"
)

In [ ]:
xgb_model.fit({"train": train_input, "validation": validation_input}, wait=True)